### Initialization and general setup
This file is automatically generated using equi2py. Created on 2023-11-29 10:08:25



In [12]:
import os
from pathlib import Path
import numpy as np
from collections import OrderedDict

import chemapp as ca
from chemapp.friendly import (
    Info,
    StreamCalculation as casc,
    ThermochemicalSystem as cats,
    EquilibriumCalculation as caec,
    Units as cau,
    TemperatureUnit,
    PressureUnit,
    VolumeUnit,
    AmountUnit,
    EnergyUnit,
    Status,
)
from chemapp.core import (
    LimitVariable,
    ChemAppError,
)

### Load data file


In [13]:
cst_file = Path("HotMetalLiquid.cst")
# load the database file
cats.load(cst_file)

# set the limits for target calculations, if deviating from the standards as below:
# T = 250.0 - 10000.0 K
# P = 1e-35 - 1e08 bar
# V = 1e-08 - 1e35 dm3
# note that these limits are always set in the above units
ca.basic.tqclim(LimitVariable.TLOW, 100.0)

### Units setup
It is possible to change the units before retrieving results, or in between making multiple inputs. However, mixed

unit input is currently not generated automatically.



In [14]:
cau.set(
    T=TemperatureUnit.C,
    P=PressureUnit.atm,
    V=VolumeUnit.dm3,
    A=AmountUnit.tonne,
    E=EnergyUnit.J,
)

In [18]:
import pandas as pd

# reading the csv file, setting the index to the index used in the csv file.
# remember to configure and convert units, if needed.
input_data = pd.read_csv("hotmetal.csv").set_index("ID")

### Calculation


In [19]:
# setting general boundary conditions of the calculation(s)
casc.set_eq_P(1.0)
casc.create_st(name="#1", T=1300, P=1)
casc.create_st(name="#2", T=25, P=1)
A = 0.0
A_range = [0.0]


# we keep the results in an OrderedDict, to access by variable value
# if wanted, the .values() attribute of results can be easily used like a list.
results = OrderedDict()

# iteration over the rows - since ID is set as index
 
for ID, row_data in input_data.iterrows():
    # the order and association of the table values is inferred (csv file header)
    total_mass, T, Fe, C, Si, Mn, P, S = row_data
    for A in A_range:

        # for each row, temperature was different
        casc.set_eq_T(T)

        # set incoming amounts
        # since this is a calculation with initial conditions, input needs to be made
        # using phase constituents

        casc.set_IA_pc("#1", "Fe_bcc(s)", "Fe_bcc(s)", Fe)
        casc.set_IA_pc("#1", "C_Graphite(s)", "C_Graphite(s)", C)
        casc.set_IA_pc("#1", "Si_solid(s)", "Si_solid(s)", Si)
        casc.set_IA_pc("#1", "Mn_Solid_Alpha(s)", "Mn_Solid_Alpha(s)", Mn)
        casc.set_IA_pc("#1", "P_Solid_(white)(s)", "P_Solid_(white)(s)", P)
        casc.set_IA_pc("#1", "S_alpha_orthorhombic_(s)", "S_alpha_orthorhombic_(s)", S)
        
        # the variable 'A' is the amount of oxygen, which is still fixed at 0.0
        casc.set_IA_pc("#2", "gas_ideal", "O2", A)

        try:
            # calculate
            casc.calculate_eq()
            # fetch results
            results[(ID, A)] = casc.get_result_object()
        except ChemAppError as c_err:
            print(f"[<A> = { A }] ChemApp Error {c_err.errno}:")
            print(c_err)
            # change this if you want to capture a calculation error
            # maybe input a dummy value (or other, adequate replacements?):
            # results[A] = None
            pass

In [23]:
# inspecting the results dictionary, we see the expected 5 calculations - one
# for each row of the table and all of them for a fixed amount of oxygen (0.0)
print(results.keys())

odict_keys([(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)])


In [29]:
# now, using both ID and the amount of oxygen (0.0), we can still directly
# access properties of the result object(s).
print(f"Calculated enthalpy: {results[(2, 0.0)].H:10.6e} J")

Calculated enthalpy: 1.260133e+11 J


In [20]:
# we want to 'pick' the Fe-liq phase status from the result objects, and use it
# as input to following calculations. Using `create_stream` yields 'StreamState'
# classes, which are exactly for that usecase.
iron_liquid_streams = OrderedDict()
for combined_ID, calculation_result in results.items():
    table_ID, amount_O2 = combined_ID
    iron_liquid_streams[table_ID] = calculation_result.create_stream(
        name="Iron Liquid", phs_include=["Fe-liq"]
    )

In [30]:
# Inspecting the StreamState object does not give different numbers than the
# full equilibrium calculation, because the Fe-liq phase is the only stable
# phase.
print(iron_liquid_streams[2])

Stream State
---------------------------------------------------------------
Class:       chemapp.core.StreamState

Name:        Iron Liquid
T:           1.3578E+03 C
P:           1.0000E+00 atm
Cp:          7.9641E+07 J/C
H:           1.2601E+11 J
S:           1.8816E+08 J/C
G:           -1.8087E+11 J
V:           0.0000E+00 dm3
---------------------------------------------------------------
Phase                     Constituent                         A
                                                          tonne
---------------------------------------------------------------
Fe-liq                    Fe                         9.5020E+01
Fe-liq                    C                          3.9500E+00
Fe-liq                    Mn                         4.0000E-01
Fe-liq                    P                          1.0100E-01
Fe-liq                    S                          1.9000E-02
Fe-liq                    Si                         5.1000E-01
----------------------------